# Resources

https://www.slickremix.com/docs/get-api-key-for-youtube/

https://developers.google.com/youtube/v3/docs/?apix=true#Search

Steps:
- Make API calls with Python requests library
- Collected JSON data from the response
- Stored data in pandas DataFrame

In [3]:
# import libraries
import requests
import pandas as pd
import time

In [4]:
# Keys
API_KEY = ''
CHANNEL_ID = "UCW8Ews7tdKKkBT6GdtQaXvQ"

In [9]:
# make API call (Example)
# response = requests.get('https://api.github.com').json()
# response

In [77]:
# make API call
pageToken= ''
url = 'https://www.googleapis.com/youtube/v3/search?key='+API_KEY+'&channelId='+CHANNEL_ID+'&part=snippet,id&order=date&maxResults=10000'+pageToken

response = requests.get(url).json()

In [78]:
response['items'][0]

{'kind': 'youtube#searchResult',
 'etag': 'GPD-F6lDd_CIdFDY4HJT48kk_64',
 'id': {'kind': 'youtube#video', 'videoId': '8IIryKHheYw'},
 'snippet': {'publishedAt': '2022-10-18T01:00:03Z',
  'channelId': 'UCW8Ews7tdKKkBT6GdtQaXvQ',
  'title': '(Part 3) How to Formulate the Approach for Solving SQL Interview Questions',
  'description': 'Ready to prepare for your SQL interview? Learn about how to formulate an effective approach for solving SQL interview questions ...',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/8IIryKHheYw/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/8IIryKHheYw/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/8IIryKHheYw/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'StrataScratch',
  'liveBroadcastContent': 'none',
  'publishTime': '2022-10-18T01:00:03Z'}}

In [28]:
for video in response['items']:
  if video['id']['kind'] == 'youtube#video':
    video_id = video['id']['videoId']
    video_title = video['snippet']['title']
    video_title = str(video_title).replace('&amp;','')
    upload_date = video['snippet']['publishedAt'].split('T')[0]
    # print(video_id)
    # print(video_title)
    # print(upload_date)

In [22]:
url_video_stats = 'https://www.googleapis.com/youtube/v3/videos?id='+video_id+'&part=statistics&key='+API_KEY

response_video_stats = requests.get(url_video_stats).json() 

In [23]:
response_video_stats

{'kind': 'youtube#videoListResponse',
 'etag': 'CAddcITptfRbe2n8KgFd0YWN7Nk',
 'items': [{'kind': 'youtube#video',
   'etag': '6ym9O3AvaoU0CdW1yibuuzWy1aA',
   'id': 'VYeevsVj4fU',
   'statistics': {'viewCount': '26866',
    'likeCount': '758',
    'favoriteCount': '0',
    'commentCount': '87'}}],
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 1}}

In [26]:
view_count = response_video_stats['items'][0]['statistics']['viewCount']
like_count = response_video_stats['items'][0]['statistics']['likeCount']
comment_count = response_video_stats['items'][0]['statistics']['commentCount']

In [27]:
view_count

'26866'

In [81]:
df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count', 'like_count', 'comment_count'])

In [82]:
for video in response['items']:
  if video['id']['kind'] == 'youtube#video':
    video_id = video['id']['videoId']
    video_title = video['snippet']['title']
    video_title = str(video_title).replace('&amp;','')
    upload_date = video['snippet']['publishedAt'].split('T')[0]

    # api call to collect video stats for each video
    url_video_stats = 'https://www.googleapis.com/youtube/v3/videos?id='+video_id+'&part=statistics&key='+API_KEY
    response_video_stats = requests.get(url_video_stats).json() 
    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']

    # save data in pandas df
    df = df.append({'video_id': video_id, 'video_title': video_title, 'upload_date': upload_date,
                    'view_count': view_count, 'like_count': like_count, 'comment_count': comment_count}, ignore_index=True)

In [83]:
df.head()
len(df)

49

# Organize our Code

In [87]:
# get url for the channel to make api call
pageToken= ''
url = 'https://www.googleapis.com/youtube/v3/search?key='+API_KEY+'&channelId='+CHANNEL_ID+'&part=snippet,id&order=date&maxResults=10000'+pageToken

In [88]:
def get_video_stats(video_id):
  url_video_stats = 'https://www.googleapis.com/youtube/v3/videos?id='+video_id+'&part=statistics&key='+API_KEY

  # make API call
  response_video_stats = requests.get(url_video_stats).json() 
  view_count = response_video_stats['items'][0]['statistics']['viewCount']
  like_count = response_video_stats['items'][0]['statistics']['likeCount']
  comment_count = response_video_stats['items'][0]['statistics']['commentCount']
  
  return view_count, like_count, comment_count

In [89]:
def get_data(url):

  df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count', 'like_count', 'comment_count'])
  
  # make API call
  response = requests.get(url).json()
  time.sleep(1)

  for video in response['items']:
    if video['id']['kind'] == 'youtube#video':
      video_id = video['id']['videoId']
      video_title = video['snippet']['title']
      video_title = str(video_title).replace('&amp;','')
      upload_date = video['snippet']['publishedAt'].split('T')[0]

      view_count, like_count, comment_count = get_video_stats(video_id)


      df = df.append({'video_id': video_id, 'video_title': video_title, 'upload_date': upload_date,
                     'view_count': view_count, 'like_count': like_count, 'comment_count': comment_count}, ignore_index=True)
  return df

In [90]:
df = get_data(url)
df.head()

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,8IIryKHheYw,(Part 3) How to Formulate the Approach for Sol...,2022-10-18,98,2,0
1,yQSZT65WET4,Superfast Python Coding Question From Amazon,2022-10-13,707,38,1
2,OMHbYrL09io,A/B Testing (A) Interview Questions or (B) for...,2022-10-12,429,19,0
3,SGDchcWjaTA,(Part 2) Deepen Your Understanding of Data wit...,2022-10-10,493,21,0
4,9av7jQ9MA44,A/B Testing Concepts To Know For Data Science,2022-10-06,1442,87,4


In [91]:
len(df)

49

# Dataframe to AWS RDS Database

Steps:
1. Create a AWS RDS postgres database - publically accessible
2. Add a security group with an **`inbound rule`** to to Allow access from All IPv4 (or just `Google Colab` IP -> find colab's instance ip address with `!curl ipecho.net/plain` command), to connect from our PC(Colab instance) to RDS.
3. Connect to the database
4. Use cursor to use python code to execute SQL commands in the DB session
5. Here our data (rows) for existing videos, will change (some entries like view_count etc), so we have to update them as well as add new rows for new videos uploaded.

**COLAB IP WILL CHANGE, so if used in INBOUND rule needs to be updated**

In [9]:
# THIS IP WILL CHANGE
!curl ipecho.net/plain

34.121.162.191

In [10]:
!pip install psycopg2
import psycopg2 as ps
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
def connect_to_db(hostname, dbname, username, password, port):

  try:
    conn = ps.connect(host=hostname, 
                      database=dbname, 
                      user=username, 
                      password=password, 
                      port=port)
  except ps.OperationalError as e:
    raise e
  else:
    print('Connected to Database')
  return conn

In [ ]:
hostname = 'database-1.XXXXXXXX.us-east-1.rds.amazonaws.com'
dbname = ''
port = '5432'
username = 'postgres'
password = 'password'
conn = None

conn = connect_to_db(hostname, dbname, username, password, port)

Connected to Database


In [37]:
# Create Table

def create_table(curr):
  create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
    video_id VARCHAR(255) PRIMARY KEY,
    video_title TEXT NOT NULL,
    upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
    view_count INTEGER NOT NULL,
    like_count INTEGER NOT NULL,
    comment_count INTEGER NOT NULL
  )""")

  curr.execute(create_table_command)

- updating the database
- handling data for scalability
- check to see if video exists
 - if video exists perform an update
 - for new videos append entry to DB Table

In [26]:
# check if video exists

def check_if_video_exists(curr, video_id):
  query = ("""SELECT video_id FROM videos WHERE video_id = %s""")
  curr.execute(query, (video_id,))

  return curr.fetchone() is not None

In [27]:
# update row if video exists

def update_row(curr, video_id, video_title, view_count, like_count, comment_count):
  query = ("""UPDATE videos
              SET video_title = %s,
                  view_count = %s,
                  like_count = %s,
                  comment_count = %s
              WHERE video_id = %s;""")
  vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
  curr.execute(query, vars_to_update)

In [33]:
# move data from df to DB

def update_db(curr, df):
  tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count', 'like_count', 'comment_count'])

  for index, row in df.iterrows():
    if check_if_video_exists(curr, row['video_id']):
      update_row(curr, row['video_id'], row['video_title'], row['view_count'], row['like_count'], row['comment_count'])  # if video already exists we will perform an update
    else: # We will append this new video to the Table
      tmp_df = tmp_df.append(row)

  return tmp_df

In [29]:
# insert row into Table for new videos

def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, comment_count):
  insert_command = ("""INSERT INTO videos (video_id, video_title, upload_date, view_count, like_count, comment_count) 
                                    VALUES (%s,%s,%s,%s,%s,%s);""")
  
  row_to_insert = (video_id, video_title, upload_date, view_count, like_count, comment_count)
  curr.execute(insert_command, row_to_insert)

In [55]:
# append rows from df(new videos) to DB

def append_from_df_to_db(curr, df):
  for index, row in new_video_df.iterrows():
    insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['comment_count'])

In [62]:
# cursor commands allows to run SQL commnds on the database
# Allows python code to execute SQL commands in your DB session
curr = conn.cursor()

In [63]:
create_table(curr)

In [64]:
new_video_df = update_db(curr, df)

In [65]:
append_from_df_to_db(curr, new_video_df)

In [66]:
conn.commit()